### **Aluno: Leonardo Ribeiro Schaedler**

# Atividade MapReduce

Considerando o dataset detalhado a seguir, extraia o conjunto de informações solicitadas.

### Dataset dados de operacoes comerciais entre paises
- Dados relativos a transacoes comerciais entre paises ao longo do tempo
- 2 bases:
  - base.csv ~433MB (3M de eventos)
  - base_sample.csv ~4MB (35k de eventos)
- 4.4M de instâncias

### Formato da base

|  # | Nome do campo        	| Descrição                                     	|
|----|----------------------	|-----------------------------------------------	|
|  0 | 	País	País envolvido na transação comercial|
|  1 | 	Ano	Ano em que a transação foi efetuada|
|  2 | 	Código	Código da mercadoria|
|  3 | 	Mercadoria 	Descrição da mercadoria|
|  4 | 	Fluxo	Fluxo, e.g. Exportação ou Importação|
|  5 | 	Valor	Valor em dólares|
|  6 | 	Peso	Peso da mercadoria|
|  7 | 	Unidade 	Unidade de medida da mercadoria, e.g. Quantidade de itens|
|  8 | 	Quantidade	Quantidade conforme a unidade especificada da mercadoria|
|  9 | 	Categoria	Categoria da mercadoria, e.g. Produto Animal|


### Informações a serem extraídas:

1.	País com a maior quantidade de transações comerciais efetuadas;
2.	Mercadoria com a maior quantidade de transações comerciais no Brasil (como a base de dados está em inglês utilize Brazil, com Z, utilize a função “contains” do Java);
3.	Quantidade de transações financeiras realizadas por ano;
4.	Mercadoria com maior quantidade de transações financeiras;
5.	Mercadoria com maior quantidade de transações financeiras em 2016;
6.	Mercadoria com maior quantidade de transações financeiras em 2016, no Brasil (como a base de dados está em inglês utilize Brazil, com Z);
7.	Mercadoria com maior total de peso, de acordo com todas transações comerciais;
8.	Mercadoria com maior total de peso, de acordo com todas transações comerciais, separadas de acordo com o ano;
9.	Média de peso por mercadoria, separadas de acordo com o ano;
10.	Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), separadas de acordo com o ano;
11.	Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), em relação ao fluxo, separadas de acordo com o ano;
12.	Preco medio das commodities de acordo com o ano
13.	Valores maximos, minimos e médios de cada tipo de mercadoria por ano
14.	Pais com o maior preco da commodity em fluxos do tipo Export
15.	Quantidade de transações comerciais de acordo com o fluxo, de acordo com o ano;

In [1]:
#Instala ambiente local do MapReduce
!pip --quiet install mrjob
#Efetua download da base completa (400MB)  https://drive.google.com/file/d/1LdeDR5wKP15kUywI7491OmKzdMzVgO-c/view?usp=sharing
!gdown 1LdeDR5wKP15kUywI7491OmKzdMzVgO-c
#Efetua download da base parcial (4MB)   https://drive.google.com/file/d/1CBi-jpBlJrKX4BmWMO0bM-DjLeLXrnDm/view?usp=sharing
!gdown 1CBi-jpBlJrKX4BmWMO0bM-DjLeLXrnDm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 6.7 MB/s eta 0:00:00
Downloading...
From: https://drive.google.com/uc?id=1LdeDR5wKP15kUywI7491OmKzdMzVgO-c
To: /content/base.csv
100% 444M/444M [00:04<00:00, 105MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CBi-jpBlJrKX4BmWMO0bM-DjLeLXrnDm
To: /content/base_sample.csv
100% 4.06M/4.06M [00:00<00:00, 192MB/s]


### Informação 1
 - País com a maior quantidade de transações comerciais efetuadas;

In [2]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        pais = campos[0]
        yield(pais, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + 1
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Writing pratica.py


Teste prévio com base parcial (Top 1 = "Canada"	936)

In [3]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.183825.766637
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.183825.766637...

Top 10 Entradas da atividade
"Canada"	936
"France"	762
"Germany"	740
"Netherlands"	717
"Greece"	687
"Italy"	655
"Denmark"	646
"Czech Rep."	643
"Malaysia"	640
"Austria"	614


Teste com base completa (Top 1 = "Australia"	89487)

In [4]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.183826.937149
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.183826.937149...

Top 10 Entradas da atividade
"Australia"	89487
"Canada"	69468
"Austria"	58683
"Argentina"	57729
"China, Hong Kong SAR"	55535
"Brazil"	54356
"Denmark"	49905
"Belgium"	49525
"China"	49275
"France"	46052


### Informação 2
 - Mercadoria com a maior quantidade de transações comerciais no Brasil (como a base de dados está em inglês utilize Brazil, com Z)

In [53]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        country = campos[0]
        if country == "Brazil":
          comm = campos[3]
          yield(comm, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [54]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.191602.357923
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.191602.357923...

Top 10 Entradas da atividade
"Horses, live except pure-bred breeding"	64
"Animals, live, except farm animals"	56
"Bovine animals, live pure-bred breeding"	56
"Fowls, live domestic < 185 grams"	56
"Bovine animals, live, except pure-bred breeding"	54
"Horses, live pure-bred breeding"	54
"Swine, live pure-bred breeding"	52
"Sheep, live"	47
"Poultry, live except domestic fowls, < 185 grams"	32
"Goats, live"	30


Teste com base completa

In [52]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.191100.937858
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.191100.937858...

Top 10 Entradas da atividade
"Industrial fatty alcohols"	84
"Cigarette or pipe tobacco and tobacco substitute mixe"	69
"Synthetic organic pigments and preps based thereon"	69
"Beauty, makeup and suntan preparations nes"	68
"Chocolate/cocoa food preparations nes"	68
"Chocolate, cocoa preps, block, slab, bar, filled, >2k"	68
"Essential oils, nes"	68
"Hair preparations, nes"	68
"Hair shampoos"	68
"Jams, fruit jellies, purees and pastes, except citrus"	68


### Informação 3
 - Quantidade de transações financeiras realizadas por ano;

In [55]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        yield(year, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [56]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.192121.145999
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.192121.145999...

Top 10 Entradas da atividade
"2012"	1628
"2013"	1621
"2003"	1580
"2010"	1571
"2014"	1568
"2011"	1566
"2001"	1557
"2015"	1551
"2000"	1549
"2007"	1538


Teste com base completa

In [57]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.192122.188615
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.192122.188615...

Top 10 Entradas da atividade
"2012"	137457
"2011"	137418
"2010"	137198
"2009"	136645
"2006"	135996
"2007"	135855
"2013"	135501
"2005"	133867
"2014"	133270
"2008"	133210


### Informação 4
 - Mercadoria com maior quantidade de transações financeiras;

In [58]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        comm = campos[3]
        yield(comm, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [59]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.192636.935567
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.192636.935567...

Top 10 Entradas da atividade
"Animals, live, except farm animals"	4413
"Fowls, live domestic < 185 grams"	3383
"Horses, live except pure-bred breeding"	3331
"Horses, live pure-bred breeding"	2971
"Bovine animals, live, except pure-bred breeding"	2758
"Poultry, live except domestic fowls, < 185 grams"	2494
"Bovine animals, live pure-bred breeding"	2490
"Sheep, live"	2287
"Goats, live"	2002
"Poultry, live except domestic fowls, > 185 grams"	1991


Teste com base completa

In [60]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.192638.166402
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.192638.166402...

Top 10 Entradas da atividade
"Food preparations nes"	8048
"Sugar confectionery not chewing gum, no cocoa content"	7764
"Sauces nes, mixed condiments, mixed seasoning"	7693
"Cigarettes containing tobacco"	7620
"Communion wafers, rice paper, bakers wares nes"	7530
"Sweet biscuits, waffles and wafers"	7402
"Chocolate/cocoa food preparations nes"	7267
"Refined sugar, in solid form, nes, pure sucrose"	7258
"Animal feed preparations nes"	7242
"Coffee extracts, essences, concentrates, preparations"	7043


### Informação 5
 - Mercadoria com maior quantidade de transações financeiras em 2016

In [61]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        if year == "2016":
          comm = campos[3]
          yield(comm, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [62]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.192732.537573
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.192732.537573...

Top 10 Entradas da atividade
"Animals, live, except farm animals"	161
"Horses, live except pure-bred breeding"	121
"Fowls, live domestic < 185 grams"	119
"Horses, live pure-bred breeding"	107
"Bovine animals, live, except pure-bred breeding"	98
"Bovine animals, live pure-bred breeding"	89
"Sheep, live"	86
"Swine, live pure-bred breeding"	71
"Goats, live"	70
"Poultry, live except domestic fowls, > 185 grams"	69


Teste com base completa

In [63]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.192733.651406
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.192733.651406...

Top 10 Entradas da atividade
"Food preparations nes"	278
"Cigarettes containing tobacco"	272
"Communion wafers, rice paper, bakers wares nes"	269
"Sauces nes, mixed condiments, mixed seasoning"	268
"Sugar confectionery not chewing gum, no cocoa content"	267
"Sweet biscuits, waffles and wafers"	267
"Chocolate/cocoa food preparations nes"	258
"Malt extract & limited cocoa pastrycooks products nes"	251
"Animal feed preparations nes"	249
"Coffee extracts, essences, concentrates, preparations"	249


### Informação 6
 - Mercadoria com maior quantidade de transações financeiras em 2016, no Brasil (como a base de dados está em inglês utilize Brazil, com Z);

In [64]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        country = campos[0]
        year = campos[1]
        if country == "Brazil" and year == "2016":
          comm = campos[3]
          yield(comm, 1)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [65]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.192848.662170
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.192848.662170...

Top 10 Entradas da atividade
"Animals, live, except farm animals"	2
"Bovine animals, live, except pure-bred breeding"	2
"Bovine animals, live pure-bred breeding"	2
"Fowls, live domestic < 185 grams"	2
"Horses, live except pure-bred breeding"	2
"Horses, live pure-bred breeding"	2
"Swine, live pure-bred breeding"	2
"Fowls, live domestic > 185 grams"	1
"Sheep, live"	1
"Swine, live except pure-bred breeding < 50 kg"	1


Teste com base completa

In [66]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.192849.972856
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.192849.972856...

Top 10 Entradas da atividade
"Acid and mordant dyes and preparations based thereon"	2
"Acrylic or vinyl polymer paint or varnish, non-aqueou"	2
"Acrylic & vinyl polymer based paint, varnish, in wate"	2
"Adrenal cortical hormones, in dosage"	2
"Agar-agar"	2
"Agarbatti, odorifers operated by burning"	2
"Agglomerated dolomite (including tarred dolomite)"	2
"Alcoholic liqueurs nes"	2
"Alkali metals other than sodium"	2
"Alkaloids, derivs, no antibiotics, hormones, in dosag"	2


### Informação 7
 - Mercadoria com maior total de peso, de acordo com todas transações comerciais;

In [74]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        comm = campos[3]
        weight = float(campos[6])
        yield(comm, weight)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [75]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.194623.273386
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.194623.273386...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding"	72350442141.0
"Swine, live except pure-bred breeding > 50 kg"	38798669146.0
"Fowls, live domestic > 185 grams"	18665789461.0
"Sheep, live"	13771569405.0
"Swine, live except pure-bred breeding < 50 kg"	12228873816.0
"Bovine animals, live pure-bred breeding"	7733896254.0
"Poultry, live except domestic fowls, > 185 grams"	4546972430.0
"Horses, live except pure-bred breeding"	2396204313.0
"Animals, live, except farm animals"	1469485365.0
"Fowls, live domestic < 185 grams"	1404224958.0


Teste com base completa

In [76]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.194721.635943
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.194721.635943...

Top 10 Entradas da atividade
"Petroleum oils, oils from bituminous minerals, crude"	46002412921988.0
"Iron ore, concentrate, not iron pyrites,unagglomerate"	34878419167261.0
"Ice, snow and potable water not sweetened or flavoure"	25759966772196.0
"Bituminous coal, not agglomerated"	21959118134095.0
"Oils petroleum, bituminous, distillates, except crude"	19216117424056.0
"Natural gas in gaseous state"	10696575625293.0
"Iron ore, concentrate, not iron pyrites, agglomerated"	7116880502870.0
"Coal except anthracite or bituminous, not agglomerate"	6161036788891.0
"Wheat except durum wheat, and meslin"	5452684784869.0
"Natural gas, liquefied"	5297769728104.0


### Informação 8
 - Mercadoria com maior total de peso, de acordo com todas transações comerciais, separadas de acordo com o ano;

In [80]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        comm = campos[3]
        weight = float(campos[6])
        yield(comm + " " + year, weight)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      yield(chave, valor)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [81]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.201827.525244
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.201827.525244...

Top 10 Entradas da atividade
"Bovine animals, live, except pure-bred breeding 2011"	3790561468.0
"Bovine animals, live, except pure-bred breeding 2010"	3725385865.0
"Bovine animals, live, except pure-bred breeding 2009"	3667337654.0
"Bovine animals, live, except pure-bred breeding 2007"	3582928269.0
"Bovine animals, live, except pure-bred breeding 2012"	3532005526.0
"Bovine animals, live, except pure-bred breeding 2006"	3474206442.0
"Bovine animals, live, except pure-bred breeding 2008"	3438256284.0
"Bovine animals, live, except pure-bred breeding 2013"	3438146811.0
"Bovine animals, live, except pure-bred breeding 2015"	3430878298.0
"Bovine animals, live, except pure-bred breeding 2005"	3358493111.0


Teste com base completa

In [82]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.201829.194176
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.201829.194176...

Top 10 Entradas da atividade
"Iron ore, concentrate, not iron pyrites,unagglomerate 2016"	2706312145364.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2015"	2649415129890.0
"Petroleum oils, oils from bituminous minerals, crude 2010"	2613316703587.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2014"	2555725226600.0
"Petroleum oils, oils from bituminous minerals, crude 2011"	2547369237511.0
"Petroleum oils, oils from bituminous minerals, crude 2014"	2389423152614.0
"Petroleum oils, oils from bituminous minerals, crude 2015"	2334398373730.0
"Petroleum oils, oils from bituminous minerals, crude 2012"	2319929864134.0
"Iron ore, concentrate, not iron pyrites,unagglomerate 2013"	2300741188739.

### Informação 9
 - Média de peso por mercadoria, separadas de acordo com o ano;

In [89]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      try:
        campos = linha.split(';')
        year = campos[1]
        comm = campos[3]
        weight = float(campos[6])
        yield(comm + " " + year, weight)
      except:
        pass

    def reducer(self, chave, valores):
      valor = 0
      for v in valores:
        valor = valor + v
      avg = valor / len(list(valores))
      yield(chave, avg)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [90]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.204144.831936

Error while reading from /tmp/pratica.root.20231019.204144.831936/step/000/reducer/00000/input:

Traceback (most recent call last):
  File "/content/pratica.py", line 23, in <module>
    WordCount.run()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/runner.py", line 503, in run
    self._run()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/sim.py", line 161, in _run
    self._run_step(step, step_num)
  File "/usr/local/lib/python3.10/dist-packages/mrjob/sim.py", line 170, in _run_step
    self._r

Teste com base completa

In [28]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184324.381466
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184324.381466...

Top 10 Entradas da atividade
"nada"	1


### Informação 10
 - Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), separadas de acordo com o ano;

In [29]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      yield('nada', 1)

    def reducer(self, chave, valores):
      yield('nada', 1)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [30]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184359.963361
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184359.963361...

Top 10 Entradas da atividade
"nada"	1


Teste com base completa

In [31]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184400.889685
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184400.889685...

Top 10 Entradas da atividade
"nada"	1


### Informação 11
 - Média de peso por mercadoria comercializadas no Brasil (como a base de dados está em inglês utilize Brazil, com Z), em relação ao fluxo, separadas de acordo com o ano;

In [32]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      yield('nada', 1)

    def reducer(self, chave, valores):
      yield('nada', 1)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [33]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184435.147380
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184435.147380...

Top 10 Entradas da atividade
"nada"	1


Teste com base completa

In [34]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184436.569785
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184436.569785...

Top 10 Entradas da atividade
"nada"	1


### Informação 12
 - Preco medio das commodities de acordo com o ano

In [35]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      yield('nada', 1)

    def reducer(self, chave, valores):
      yield('nada', 1)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [36]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184511.493436
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184511.493436...

Top 10 Entradas da atividade
"nada"	1


Teste com base completa

In [37]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184512.422626
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184512.422626...

Top 10 Entradas da atividade
"nada"	1


### Informação 13
 - Valores maximos, minimos e médios de cada tipo de mercadoria por ano

In [38]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      yield('nada', 1)

    def reducer(self, chave, valores):
      yield('nada', 1)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [39]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184547.817202
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184547.817202...

Top 10 Entradas da atividade
"nada"	1


Teste com base completa

In [40]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184548.744269
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184548.744269...

Top 10 Entradas da atividade
"nada"	1


### Informação 14
 - Pais com o maior preco da commodity em fluxos do tipo Export

In [41]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      yield('nada', 1)

    def reducer(self, chave, valores):
      yield('nada', 1)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [42]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184624.448598
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184624.448598...

Top 10 Entradas da atividade
"nada"	1


Teste com base completa

In [43]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184625.364487
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184625.364487...

Top 10 Entradas da atividade
"nada"	1


### Informação 15
 - Quantidade de transações comerciais de acordo com o fluxo, de acordo com o ano

In [44]:
%%file pratica.py
from mrjob.job import MRJob
import sys

class WordCount(MRJob):
    def mapper(self, _, linha):
      yield('nada', 1)

    def reducer(self, chave, valores):
      yield('nada', 1)

if __name__ == '__main__':
     WordCount.run()

Overwriting pratica.py


Teste prévio com base parcial

In [45]:
!rm -rf /content/atividade
!python pratica.py /content/base_sample.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184701.391508
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184701.391508...

Top 10 Entradas da atividade
"nada"	1


Teste com base completa

In [46]:
!rm -rf /content/atividade
!python pratica.py /content/base.csv --output-dir=/content/atividade
print('\nTop 10 Entradas da atividade')
!cat /content/atividade/* | sort -k2,2rn -t$'\t' | head -10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/pratica.root.20231019.184702.437687
job output is in /content/atividade
Removing temp directory /tmp/pratica.root.20231019.184702.437687...

Top 10 Entradas da atividade
"nada"	1
